# 모델 vs Stockfish 대결

학습된 ChessCNN 모델과 Stockfish 엔진을 대결시킵니다.

## 실행 순서
1. 모델 로드
2. Stockfish 엔진 초기화
3. 게임 진행 (모델 vs Stockfish)
4. 결과 집계

In [12]:
import torch
import torch.nn.functional as F
import numpy as np
import chess
import chess.engine
import chess.pgn
from pathlib import Path
from tqdm import tqdm
import time
from datetime import datetime

from chess_model import ChessCNN
from preprocessing import (
    board_to_tensor,
    legal_move_mask,
    action_index_to_move,
    move_to_action_index
)

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

사용 장치: cuda
  GPU: NVIDIA GeForce RTX 5060 Ti
  GPU 메모리: 15.93 GB


## 모델 로드

In [13]:
# RL 모델 로드
MODEL_PATH = Path("models/best_chess_cnn.pth")

if MODEL_PATH.exists():
    checkpoint = torch.load(MODEL_PATH, map_location=device)
    model = ChessCNN(num_channels=256).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f"✅ 모델 로드 완료!")
    print(f"   Iteration: {checkpoint.get('iteration', 'N/A')}")
    print(f"   Loss: {checkpoint.get('loss', 'N/A')}")
    print(f"   Win Rate: {checkpoint.get('win_rate', 'N/A')}")
else:
    raise FileNotFoundError(f"모델 파일을 찾을 수 없습니다: {MODEL_PATH}")

print(f"\n모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

✅ 모델 로드 완료!
   Iteration: N/A
   Loss: N/A
   Win Rate: N/A

모델 파라미터 수: 9,314,433


## Stockfish 엔진 초기화

In [14]:
# Stockfish 경로 설정

STOCKFISH_PATH = "/usr/games/stockfish"

print(f"시도할 경로: {STOCKFISH_PATH}")

try:
    engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)
    print(f"✅ Stockfish 엔진 초기화 완료!")
    print(f"   경로: {STOCKFISH_PATH}")
    
    # Stockfish 정보 확인
    info = engine.id
    print(f"   이름: {info.get('name', 'Unknown')}")
    print(f"   저자: {info.get('author', 'Unknown')}")
except FileNotFoundError as e:
    print(f"❌ Stockfish 파일을 찾을 수 없습니다: {e}")
    print(f"\n파일 탐색기에서 stockfish-windows-x86-64-avx2.exe 파일을 찾아서")
    print(f"전체 경로를 복사해서 STOCKFISH_PATH에 붙여넣으세요.")
    raise
except Exception as e:
    print(f"❌ Stockfish 초기화 실패: {e}")
    print(f"\nStockfish를 설치하거나 경로를 확인하세요.")
    print(f"다운로드: https://stockfishchess.org/download/")
    raise

시도할 경로: /usr/games/stockfish
✅ Stockfish 엔진 초기화 완료!
   경로: /usr/games/stockfish
   이름: Stockfish 16
   저자: the Stockfish developers (see AUTHORS file)


## 모델 행동 선택 함수

In [15]:
@torch.no_grad()
def model_select_move(board: chess.Board, temperature=0.1):
    """
    모델이 현재 보드에서 수를 선택합니다.
    
    Args:
        board: chess.Board 객체
        temperature: 탐색 온도 (낮을수록 greedy)
    
    Returns:
        move: 선택된 chess.Move
    """
    # 보드를 텐서로 변환
    state = board_to_tensor(board)
    mask = legal_move_mask(board)
    
    state_tensor = torch.from_numpy(state).unsqueeze(0).to(device)
    mask_tensor = torch.from_numpy(mask).unsqueeze(0).to(device)
    
    # 모델 추론
    policy_logits, _ = model(state_tensor, mask_tensor)
    policy_logits = policy_logits.squeeze(0)  # (4096,)
    
    # 불법 수 마스킹
    masked_logits = policy_logits.clone()
    masked_logits[~mask_tensor.squeeze(0).bool()] = float('-inf')
    
    # Temperature 적용
    if temperature != 1.0:
        masked_logits = masked_logits / temperature
    
    # Softmax로 확률 계산
    probs = F.softmax(masked_logits, dim=-1)
    
    # 확률적 샘플링
    action_idx = torch.multinomial(probs, 1).item()
    
    # 액션 인덱스를 Move로 변환
    move = action_index_to_move(action_idx)
    
    # 프로모션 처리 (QUEEN만 허용)
    if move.promotion is None:
        # 합법 수 중에서 같은 from/to를 가진 수 찾기
        for legal_move in board.legal_moves:
            if (legal_move.from_square == move.from_square and 
                legal_move.to_square == move.to_square):
                if legal_move.promotion is not None:
                    # 프로모션인 경우 Queen 승격
                    return chess.Move(move.from_square, move.to_square, promotion=chess.QUEEN)
                return legal_move
    
    return move

## 단일 게임 플레이

In [16]:
def play_game_model_vs_stockfish(model_is_white=True, stockfish_time=0.1, max_moves=200, temperature=0.1):
    """
    모델과 Stockfish가 한 게임을 플레이합니다.
    
    Args:
        model_is_white: 모델이 백(True)인지 흑(False)인지
        stockfish_time: Stockfish가 생각할 시간 (초)
        max_moves: 최대 수 제한
        temperature: 모델 탐색 온도
    
    Returns:
        result: 게임 결과 (1=백승, -1=흑승, 0=무승부)
        moves: 게임 수열
        final_board: 최종 보드
    """
    board = chess.Board()
    moves = []
    move_count = 0
    
    while not board.is_game_over() and move_count < max_moves:
        if board.turn == chess.WHITE:
            # 백 차례
            if model_is_white:
                # 모델 차례
                move = model_select_move(board, temperature=temperature)
            else:
                # Stockfish 차례
                result = engine.play(board, chess.engine.Limit(time=stockfish_time))
                move = result.move
        else:
            # 흑 차례
            if not model_is_white:
                # 모델 차례
                move = model_select_move(board, temperature=temperature)
            else:
                # Stockfish 차례
                result = engine.play(board, chess.engine.Limit(time=stockfish_time))
                move = result.move
        
        board.push(move)
        moves.append(move)
        move_count += 1
    
    # 게임 결과
    if move_count >= max_moves:
        result = 0.0  # 무승부
    else:
        game_result = board.result()
        if game_result == "1-0":
            result = 1.0  # 백 승
        elif game_result == "0-1":
            result = -1.0  # 흑 승
        else:
            result = 0.0  # 무승부
    
    return result, moves, board

## 여러 게임 대결

In [17]:
# 대결 설정
NUM_GAMES = 10  # 총 게임 수
STOCKFISH_TIME = 0.1  # Stockfish 생각 시간 (초)
TEMPERATURE = 0.1  # 모델 탐색 온도 (낮을수록 greedy)
MAX_MOVES = 200  # 최대 수 제한

print(f"대결 설정:")
print(f"  총 게임 수: {NUM_GAMES}")
print(f"  Stockfish 시간: {STOCKFISH_TIME}초")
print(f"  모델 온도: {TEMPERATURE}")
print(f"  최대 수: {MAX_MOVES}")
print()

# 결과 저장
model_as_white_wins = 0
model_as_white_losses = 0
model_as_white_draws = 0

model_as_black_wins = 0
model_as_black_losses = 0
model_as_black_draws = 0

all_games = []

# 게임 진행
for game_idx in tqdm(range(NUM_GAMES), desc="게임 진행"):
    # 모델이 백/흑 번갈아 플레이
    model_is_white = (game_idx % 2 == 0)
    
    result, moves, final_board = play_game_model_vs_stockfish(
        model_is_white=model_is_white,
        stockfish_time=STOCKFISH_TIME,
        max_moves=MAX_MOVES,
        temperature=TEMPERATURE
    )
    
    # 모델 관점에서 결과 계산
    if model_is_white:
        if result > 0:
            model_as_white_wins += 1
        elif result < 0:
            model_as_white_losses += 1
        else:
            model_as_white_draws += 1
    else:
        if result < 0:  # 흑 승 = 모델 승
            model_as_black_wins += 1
        elif result > 0:  # 백 승 = 모델 패
            model_as_black_losses += 1
        else:
            model_as_black_draws += 1
    
    all_games.append({
        'game_idx': game_idx + 1,
        'model_is_white': model_is_white,
        'result': result,
        'num_moves': len(moves),
        'final_board': final_board
    })
    
    # 진행 상황 출력 (매 5게임마다)
    if (game_idx + 1) % 5 == 0:
        total_wins = model_as_white_wins + model_as_black_wins
        total_losses = model_as_white_losses + model_as_black_losses
        total_draws = model_as_white_draws + model_as_black_draws
        total_score = total_wins + total_draws * 0.5
        win_rate = total_score / (game_idx + 1) if (game_idx + 1) > 0 else 0
        
        print(f"\n[{game_idx + 1}/{NUM_GAMES}] 승: {total_wins}, 패: {total_losses}, 무: {total_draws}, 승률: {win_rate*100:.1f}%")

## 결과 집계

## PGN 형식 게임 출력

In [18]:
# PGN 형식으로 게임 출력
print("\n" + "=" * 60)
print("PGN 형식 게임")
print("=" * 60)

# PGN 파일 저장
pgn_file_path = Path("games_vs_stockfish.pgn")
with open(pgn_file_path, "w", encoding="utf-8") as pgn_file:
    for game in all_games:
        # 보드에서 직접 PGN 생성 (python-chess가 자동으로 처리)
        game_pgn = chess.pgn.Game.from_board(game['final_board'])
        
        # 헤더 정보만 추가
        game_pgn.headers["Event"] = "Model vs Stockfish"
        game_pgn.headers["Date"] = datetime.now().strftime("%Y.%m.%d")
        game_pgn.headers["Round"] = str(game['game_idx'])
        
        if game['model_is_white']:
            game_pgn.headers["White"] = "ChessCNN Model"
            game_pgn.headers["Black"] = "Stockfish"
        else:
            game_pgn.headers["White"] = "Stockfish"
            game_pgn.headers["Black"] = "ChessCNN Model"
        
        # 출력 및 저장
        print(f"\n[게임 {game['game_idx']}]")
        print(game_pgn, end="\n\n")
        print(game_pgn, file=pgn_file, end="\n\n")

print(f"\n✅ PGN 파일 저장 완료: {pgn_file_path.absolute()}")


PGN 형식 게임

[게임 1]
[Event "Model vs Stockfish"]
[Site "?"]
[Date "2026.02.03"]
[Round "1"]
[White "ChessCNN Model"]
[Black "Stockfish"]
[Result "0-1"]

1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. Nc3 a6 6. Bg5 e6 7. f4 h6 8. Bh4 Qb6 9. Nb3 Qe3+ 10. Qe2 Qxf4 11. Bxf6 Qxf6 12. O-O-O Nc6 13. g4 Ne5 14. Bg2 g6 15. Rhf1 Qd8 16. h3 b5 17. Kb1 b4 18. Na4 Bd7 19. Qf2 Bxa4 20. Rd4 Rb8 21. Qd2 Qb6 22. Rf6 a5 23. g5 hxg5 24. Qxg5 Bxb3 25. axb3 Qxd4 26. Rf1 a4 27. bxa4 b3 28. c3 Qd3+ 29. Ka1 Ra8 30. Rf6 Rxa4# 0-1


[게임 2]
[Event "Model vs Stockfish"]
[Site "?"]
[Date "2026.02.03"]
[Round "2"]
[White "Stockfish"]
[Black "ChessCNN Model"]
[Result "1-0"]

1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. Nc3 a6 6. Be3 e6 7. Qd2 Be7 8. f3 O-O 9. g4 b5 10. O-O-O Bb7 11. g5 Nfd7 12. h4 b4 13. Nce2 Qa5 14. Kb1 Nc6 15. Nxc6 Bxc6 16. Nd4 Bb7 17. h5 Rac8 18. g6 h6 19. Bxh6 Bf6 20. Be3 Nc5 21. Bh3 Na4 22. gxf7+ Kxf7 23. Bxe6+ Ke7 24. Bb3 Nc5 25. Nf5+ Kd7 26. Qxd6+ Ke8 27. Qe7+ Bxe7 28. Nxg7# 1-0


[게임 3]
[Event 

In [19]:
# 최종 결과 계산
total_wins = model_as_white_wins + model_as_black_wins
total_losses = model_as_white_losses + model_as_black_losses
total_draws = model_as_white_draws + model_as_black_draws
total_score = total_wins + total_draws * 0.5
win_rate = total_score / NUM_GAMES if NUM_GAMES > 0 else 0

print("=" * 60)
print("최종 결과")
print("=" * 60)
print(f"총 게임 수: {NUM_GAMES}")
print(f"\n모델이 백일 때:")
print(f"  승: {model_as_white_wins}, 패: {model_as_white_losses}, 무: {model_as_white_draws}")
print(f"\n모델이 흑일 때:")
print(f"  승: {model_as_black_wins}, 패: {model_as_black_losses}, 무: {model_as_black_draws}")
print(f"\n전체:")
print(f"  승: {total_wins}, 패: {total_losses}, 무: {total_draws}")
print(f"  승률: {win_rate*100:.1f}% ({total_score}/{NUM_GAMES})")
print("=" * 60)

최종 결과
총 게임 수: 10

모델이 백일 때:
  승: 0, 패: 5, 무: 0

모델이 흑일 때:
  승: 0, 패: 5, 무: 0

전체:
  승: 0, 패: 10, 무: 0
  승률: 0.0% (0.0/10)


## 게임 상세 정보

In [20]:
# 각 게임 상세 정보 출력
print("\n게임 상세 정보:")
print("-" * 60)
for game in all_games:
    color = "백" if game['model_is_white'] else "흑"
    if game['result'] > 0:
        result_str = "백 승"
    elif game['result'] < 0:
        result_str = "흑 승"
    else:
        result_str = "무승부"
    
    model_result = "승" if (game['model_is_white'] and game['result'] > 0) or \
                        (not game['model_is_white'] and game['result'] < 0) else \
                   "패" if (game['model_is_white'] and game['result'] < 0) or \
                        (not game['model_is_white'] and game['result'] > 0) else "무"
    
    print(f"게임 {game['game_idx']:2d}: 모델({color}) - {result_str} ({model_result}), {game['num_moves']}수")


게임 상세 정보:
------------------------------------------------------------
게임  1: 모델(백) - 흑 승 (패), 60수
게임  2: 모델(흑) - 백 승 (패), 55수
게임  3: 모델(백) - 흑 승 (패), 88수
게임  4: 모델(흑) - 백 승 (패), 39수
게임  5: 모델(백) - 흑 승 (패), 66수
게임  6: 모델(흑) - 백 승 (패), 57수
게임  7: 모델(백) - 흑 승 (패), 24수
게임  8: 모델(흑) - 백 승 (패), 91수
게임  9: 모델(백) - 흑 승 (패), 24수
게임 10: 모델(흑) - 백 승 (패), 69수


## 정리

In [21]:
# Stockfish 엔진 종료
engine.quit()
print("✅ Stockfish 엔진 종료 완료")

✅ Stockfish 엔진 종료 완료
